<a href="https://colab.research.google.com/github/arafMustavi/Chatbot/blob/master/ChatBot_Using_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup of Environment

In [1]:
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Tokenizer and Stemmer

In [2]:
def tokenize(sentence):
  return nltk.word_tokenize(sentence)

stemmer = PorterStemmer()

def stem_word(word):
  word = word.lower()
  return stemmer.stem(word)
  

Load Data and Train

In [5]:
import json

with open('/content/sample_data/intents.json','r') as f:
  intents = json.load(f)
print(intents)



{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hey :-)', 'Hello, thanks for visiting', 'Hi there, what can I do for you?', 'Hi there, how can I help?']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'items', 'patterns': ['Which items do you have?', 'What kinds of items are there?', 'What do you sell?'], 'responses': ['We sell coffee and tea', 'We have coffee and tea']}, {'tag': 'payments', 'patterns': ['Do you take credit cards?', 'Do you accept Mastercard?', 'Can I pay with Paypal?', 'Are you cash only?'], 'responses': ['We accept VISA, Mastercard and Paypal', 'We accept most major credit cards, and Paypal']}, {'tag': 'd

In [9]:
all_words = []
tags = []
xy = []

for intent in intents['intents']:
  tag = intent['tag']
  tags.append(tag)
  for pattern in intent['patterns']:
    tokens = tokenize(pattern)
    all_words.extend(tokens)
    xy.append((tokens,tag))

In [10]:
print(all_words)

['Hi', 'Hey', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Good', 'day', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Thank', "'s", 'a', 'lot', '!', 'Which', 'items', 'do', 'you', 'have', '?', 'What', 'kinds', 'of', 'items', 'are', 'there', '?', 'What', 'do', 'you', 'sell', '?', 'Do', 'you', 'take', 'credit', 'cards', '?', 'Do', 'you', 'accept', 'Mastercard', '?', 'Can', 'I', 'pay', 'with', 'Paypal', '?', 'Are', 'you', 'cash', 'only', '?', 'How', 'long', 'does', 'delivery', 'take', '?', 'How', 'long', 'does', 'shipping', 'take', '?', 'When', 'do', 'I', 'get', 'my', 'delivery', '?', 'Tell', 'me', 'a', 'joke', '!', 'Tell', 'me', 'something', 'funny', '!', 'Do', 'you', 'know', 'a', 'joke', '?']


In [11]:
ignore_words = ["?" , "," , "!" , "." , "(" , ")" ]

In [15]:
all_words = [stem_word(word) for word in all_words if word not in ignore_words ]

all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [16]:
print(tags)

['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']


In [18]:
print(all_words)

["'s", 'a', 'accept', 'anyon', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'deliveri', 'do', 'doe', 'funni', 'get', 'good', 'goodbi', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'my', 'of', 'onli', 'pay', 'paypal', 'see', 'sell', 'ship', 'someth', 'take', 'tell', 'thank', 'that', 'there', 'what', 'when', 'which', 'with', 'you']


Train Data

In [24]:
import numpy as np
X_train = np.array(X_train)
Y_train = np.array(Y_train)


NameError: ignored

In [27]:
def bag_of_words(tokenized_sentence,all_words):
  tokenized_sentence = [stem_word(word) for word in tokenized_sentence]
  bag = np.zeros(len(all_words) , dtype = np.float32)

  for (idx,w) in enumerate(all_words):
    if w in tokenized_sentence:
      bag[idx] = 1.0
  
  return bag  

In [28]:
sentence = ["Bangadesh","hello","hi"]
words = ["bang","hi", "hello", "okay","Bangladesh"]
bag = bag_of_words(sentence, words)

print(bag)

[0. 1. 1. 0. 0.]


In [ ]:
X_train = []
Y_train = []
for (pattern_sentence,tag) in xy:
  bag = bag_of_words(pattern_sentence,all_words)
  X_train.append(bag)

  label = tags.index(tags)
  Y_train.append(label)

PyTorch Begins

In [29]:
import torch
from torch.nn import nn
from torch.usils.data import Dataset,Dataloader